### 初期設定

In [4]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
from utils.data_loader import load_csv_to_db, get_df_from_db
import utils.config as cfg
import utils.visualize_dashboard as dv
import utils.calculation as dp
print("🔁 autoreload 有効化完了")
import os
import pandas as pd
import plotly.express as px
import yaml

# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw'},
 'files': {'raw': {'資産詳細': '資産詳細.csv',
   '収支詳細': '収支詳細.csv',
   '資産目標値': '資産目標値.csv'}},
 'database_path': './database',
 'database': {'finance': 'finance.db'}}

In [ ]:
csv_files = {}
for file in settings["files"]["raw"]:
    csv_files[settings["files"]["raw"][file]] = os.path.join(settings["data_path"]["raw"],"",settings["files"]["raw"][file]) 
#print(csv_files)
DB_PATH_FINANCE = os.path.join(settings["database_path"],"",settings["database"]["finance"])

### 動作確認

#### _________ダッシュボード

In [ ]:
# 1-2.資産、収益
df_asset = get_df_from_db(
    DB_PATH_FINANCE, "資産詳細", "date", None, ["資産額","トータルリターン"],"sum",set_index=True)

end_date = df_asset.index.max() + pd.DateOffset(months=1)
where_clause = f"date <= DATE('{end_date}')"
df_asset_target = get_df_from_db(
    DB_PATH_FINANCE, "資産目標値", "date", None, ["資産額", "トータルリターン"], "sum", set_index=True,
    where_clause=where_clause)

df_merged = pd.merge(df_asset, df_asset_target, left_index=True,right_index=True, how="outer", suffixes=("_実績", "_目標"))
#df_merged

In [ ]:
# 1. 総資産推移
fig = px.line(df_merged, x=df_merged.index, y=["資産額_実績", "資産額_目標"],
              title="総資産推移", labels={"date": "日付", "values":"資産額","variable":""}, template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=50,b=20))

In [ ]:
df_merged

In [ ]:
# 2. トータルリターン推移
fig = px.line(df_merged, x=df_merged.index, y=["トータルリターン_実績", "トータルリターン_目標"],
              title="トータルリターン推移", labels={"date": "日付", "values":"トータルリターン","variable":""},template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=50,b=20))

In [ ]:
# 3-6.収支
df_balance = get_df_from_db(
    DB_PATH_FINANCE, "収支詳細", "date", ["収支タイプ", "収支カテゴリー"],["金額", "目標"],
    "sum", set_index=True
)
df_balance

In [ ]:
# 3-4.一般
df_general = df_balance.query('収支タイプ == "一般収支"')

df_general = df_general.pivot_table(
    index="date", columns="収支カテゴリー",values=["金額", "目標"], aggfunc="sum")
df_general.columns = [f"{val}_{cat}" for val, cat in df_general.columns]

df_monthly = df_general.resample('ME').sum()

df_monthly["目標_収支"] = df_monthly["目標_収入"] - df_monthly["目標_支出"]
df_monthly["金額_収支"] = df_monthly["金額_収入"] - df_monthly["金額_支出"]


In [ ]:
# 3.一般収入・支出
fig = px.bar(
    df_monthly, x=df_monthly.index, y=["金額_収入", "金額_支出"],barmode='group', template='plotly_dark',
    labels={'value':'金額', 'date':'年月', 'variable':'種別'}, title = "一般収入・支出")
#fig.show()
fig.add_scatter(
    x=df_monthly.index,
    y=df_monthly['目標_収入'],
    mode='lines+markers',
    name='目標_収入',
    line=dict(color='blue', width=2)
)
fig.add_scatter(
    x=df_monthly.index,
    y=df_monthly['目標_支出'],
    mode='lines+markers',
    name='目標_支出',
    line=dict(color='orange', width=2)
)
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=50,b=20))


In [ ]:
# 4. 一般収支
fig = px.bar(
    df_monthly, x=df_monthly.index, y=["金額_収支"], template='plotly_dark',
    labels={'value':'金額', 'date':'年月', 'variable':''}, title = "一般収支"
)
fig.add_scatter(
    x=df_monthly.index,
    y=df_monthly['目標_収支'],
    mode='lines+markers',
    name='目標_収支',
    line=dict(color='orange', width=2)
)
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=50,b=20))
fig.show()


In [ ]:
# 5-6.特別
df_special = df_balance.query('収支タイプ == "特別収支"')

df_special = df_special.pivot_table(
    index="date", columns="収支カテゴリー",values=["金額", "目標"], aggfunc="sum")
df_special.columns = [f"{val}_{cat}" for val, cat in df_special.columns]

df_monthly = df_special.resample('ME').sum()

df_monthly["目標_収支"] = df_monthly["目標_収入"].cumsum() - df_monthly["目標_支出"].cumsum()
df_monthly["金額_収支"] = df_monthly["金額_収入"].cumsum() - df_monthly["金額_支出"].cumsum()

In [ ]:
# 5. 特別収入・支出推移
fig = px.bar(
    df_monthly, x=df_monthly.index, y=["金額_収入", "金額_支出"],barmode='group', template='plotly_dark',
    labels={'value':'金額', 'date':'年月', 'variable':'種別'}, title = "特別収入・支出")
#fig.show()
fig.add_scatter(
    x=df_monthly.index,
    y=df_monthly['目標_収入'],
    mode='lines+markers',
    name='目標_収入',
    line=dict(color='blue', width=2)
)
fig.add_scatter(
    x=df_monthly.index,
    y=df_monthly['目標_支出'],
    mode='lines+markers',
    name='目標_支出',
    line=dict(color='orange', width=2)
)
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=50,b=20))


In [ ]:
# 6. 特別収支
fig = px.line(
    df_monthly, x=df_monthly.index, y=["金額_収支","目標_収支"], template='plotly_dark', markers=True,
    labels={'value':'金額', 'date':'年月', 'variable':''}, title = "特別収支"
)
fig.show()


### SQLite関連

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
cur = conn.cursor()

# .csv付きのテーブルを削除
for table in ["資産詳細.csv", "収支詳細.csv", "資産目標値.csv"]:
    cur.execute(f'DROP TABLE IF EXISTS "{table}"')
    print(f"削除: {table}")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
conn.close()

print(tables)